# Data Manipulation

De in te lezen data is helaas niet publiek toegankelijk. Om deze data toch voor demo doeleinden te kunnen gebruiken heb ik 
middels deze routine de data een klein beetje aangepast. Hierbij is er op gelet om de samenhang van de data in de curves en in de tijd te behouden.


In [4]:
import pandas as pd
import numpy as np
import datetime
from pandas.tseries.offsets import DateOffset           # Gebruikt voor het berekenen van de datum van FRA contracten


import random as rd

                                  

In [5]:

def read_data():
    '''
    Lees de marktdata in.    
    Input: 
      -        
    Output: 
      Pandas Dataframe    
    '''            
    try:
    
        # Datum kolommen
        date_cols = ["rate_dt","input_dt","actual_dt",'start_term_dt','maturity_term_dt']
    
        # Omzetten datum kolommen. int_basis en exchange name zijn gewoon string, maar werden niet goed herkend
        df = pd.read_csv(r'data\rates_data_2020.csv', 
                         parse_dates = date_cols, 
                         dtype ={"int_basis":'S10', "exchange_name": 'S10'})

        
    except Exception as e:
            print("ERROR: Unable to find or access file:", e)
    
    return df

# Inlezen data
df = read_data()


In [6]:
# Toevoegen van gemiddelde en standaard afwijking 
keys = ['curve_type','rate_name','rate_type','ccy','time_band']
df_gr = df.groupby(keys).agg( {'mid' : ['mean', 'std'] } ) 

# hernoemen output kolommen
df_gr.columns = ['mean', 'std'] 

# Group by levert een multi index. Deze wordt hier weer plat geslagen
df_gr = df_gr.reset_index() 

# Berekenen aanpassing per rate

df_gr['change'] = round(df_gr['std']*(rd.random()-.5)*.2,5)

# Left join de change over alle data
dfi = df.merge(df_gr, on = keys, how = "left")

# Dagelijkse change per key
keys = ['rate_dt','curve_type','rate_name','rate_type','ccy']
df_grd = df.groupby(keys).agg(  {'mid' : ['mean', 'std'] } ) 

# hernoemen output kolommen
df_grd.columns = ['mean', 'std'] 
# Group by levert een multi index. Deze wordt hier weer plat geslagen
df_grd = df_grd.reset_index() 

df_grd['std'] = df_grd['std'].fillna(0)

df_grd['daily_change'] = round(df_grd['std']*(rd.random()-.5)*.2,5)

# Left join de change over alle data
dfi = dfi.merge(df_grd, on = keys, how = "left")



In [4]:

# aanpassen van de data
dfi['bid'] = dfi['bid']+dfi['change']+dfi['daily_change']
dfi['offer'] = dfi['bid']+dfi['change']+dfi['daily_change']
dfi['mid'] = dfi['bid']+dfi['change']+dfi['daily_change']


# Drop werk kolommen
del dfi['daily_change']
del dfi['change']
del dfi['mean_x']
del dfi['mean_y']
del dfi['std_x']
del dfi['std_y']


In [7]:
# Drop 31-dec aangezien rates niet representatief zijn

dfi = dfi[dfi['rate_dt'] != '31-DEC-2020']

dfi = dfi[dfi['rate_dt'] >= '1-DEC-2020'] # Subset data voor distributie


In [8]:
mask = dfi['curve_type']=='Capital Pr'

In [9]:

# Annonimizeer ook nog even de ISIN codes
f = []
l = dfi[mask]['rate_name'].unique()
for value in l:
    f.append(value[:2] + str(rd.randint(0, 999999999)))
    
def anonymize (ratename):
               
    index = np.where(l==ratename)
    name = f[index[0][0]]
    
    return name

In [10]:

dfi.loc[mask,'rate_name'] = dfi.loc[mask].apply(lambda x: anonymize(x['rate_name'])
                                                  , axis = 1)

In [18]:
# Inlezen hiearchien

def add_hiarchy(df, fieldnames):
    """
    Deze functie voegt een dimensie toe die is gedefinieerd in de json data.
        
    Input: 
        df -          Pandas Datafame 
        fieldnames    List of names of hiarchies. 
                      In de data folder moet een bestand in JSON format staan met deze naam      
    Output: 
      Pandas Dataframe     
    """    
    try:

        for name in fieldnames:            
            # Lezen van hiarchie-en vanuit JSON
            filename = 'data\\' + name + ".json"
            df_r = pd.read_json(filename)
            df = pd.merge(df,df_r,how = 'left')
    
                
    except Exception as e:
        print("ERROR: Unable to find or access file:", e)
    
    return df

hiarchies = ['yield_group']

dfi = add_hiarchy(dfi,hiarchies)

In [22]:

mask = ((dfi['curve_type']=='Yield' ) & 
        ( dfi['yield_group'] == 'CDS Specific'))
dfi = dfi[~mask]

In [25]:
dfi = dfi.drop(columns = ['yield_group'])

In [26]:

# Drop 2 maands depo rates in CHF 
# Dit om te laten zien hoe ik deze uitzonderingen detecteer
i = dfi[(dfi['rate_dt'] == dfi['rate_dt'].max()) &
             (dfi['rate_name'] == 'Deposit') &
             (dfi['ccy'] == 'CHF') &
             (dfi['time_band'] == '2 MONTHS')    
       ].index

dfi.drop(i , inplace=True)



In [27]:
# Write the changed data to disc
dfi.to_csv(r'data\rates_data.csv',index=False)